In [18]:
#importing libraries 
import pandas as pd
from geopy import distance
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [33]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
# load the dataset 
df = pd.read_excel('GJ12b.xltm')

In [35]:
# Display the data set
df.head()


,"Lat,Lngt",Period
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35


In [36]:
# Checking rows/cols
df.shape

(1450, 2)

In [37]:
#splitting lat/long col
df[['lat', 'long']] = df['Lat,Lngt'].str.split(',', 1, expand=True)

In [38]:
df

,"Lat,Lngt",Period,lat,long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,13.003621,79.990784
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,13.003621,79.990784
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,13.003621,79.990784
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,13.003621,79.990784
...,...,...,...,...
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438
1448,"28.039583,73.430438",21/10 11:55 : 21/10 11:57,28.039583,73.430438


In [39]:
df = df[0:1449]

In [40]:
df

,"Lat,Lngt",Period,lat,long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,13.003621,79.990784
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,13.003621,79.990784
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,13.003621,79.990784
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,13.003621,79.990784
...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,28.039583,73.430438
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438


In [41]:
df['dealer_lat'] = 27.99536

In [42]:
df['dealer_long'] = 73.31652

In [44]:
df

,"Lat,Lngt",Period,lat,long,dealer_lat,dealer_long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784,27.99536,73.31652
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,13.003621,79.990784,27.99536,73.31652
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,13.003621,79.990784,27.99536,73.31652
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,13.003621,79.990784,27.99536,73.31652
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,13.003621,79.990784,27.99536,73.31652
...,...,...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,28.039583,73.430438,27.99536,73.31652
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438,27.99536,73.31652
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438,27.99536,73.31652
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438,27.99536,73.31652


In [45]:
# checking data types for type conversion
df.dtypes

Lat,Lngt        object
Period          object
lat             object
long            object
dealer_lat     float64
dealer_long    float64
dtype: object

In [46]:
# lat / long is object type -> we must convert it to a floating point number 
df["lat"]= df["lat"].astype(float)
df["long"]= df["long"].astype(float)

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [47]:
df

,"Lat,Lngt",Period,lat,long,dealer_lat,dealer_long
0,"13.003621,79.990784",16/10 10:10 : 16/10 10:15,13.003621,79.990784,27.99536,73.31652
1,"13.003621,79.990784",16/10 10:15 : 16/10 10:20,13.003621,79.990784,27.99536,73.31652
2,"13.003621,79.990784",16/10 10:20 : 16/10 10:25,13.003621,79.990784,27.99536,73.31652
3,"13.003621,79.990784",16/10 10:25 : 16/10 10:30,13.003621,79.990784,27.99536,73.31652
4,"13.003621,79.990784",16/10 10:30 : 16/10 10:35,13.003621,79.990784,27.99536,73.31652
...,...,...,...,...,...,...
1444,"28.039583,73.430438",21/10 11:35 : 21/10 11:40,28.039583,73.430438,27.99536,73.31652
1445,"28.039583,73.430438",21/10 11:40 : 21/10 11:45,28.039583,73.430438,27.99536,73.31652
1446,"28.039583,73.430438",21/10 11:45 : 21/10 11:50,28.039583,73.430438,27.99536,73.31652
1447,"28.039583,73.430438",21/10 11:50 : 21/10 11:55,28.039583,73.430438,27.99536,73.31652


In [48]:
import numpy as np

### Calculating straight line distance using haversine method

In [49]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [50]:
dealer_lat = 27.99536
dealer_lon = 73.31652
distances_km = []
for row in df.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat, dealer_lon, row.lat, row.long)
   )

In [51]:
df['Distance'] = distances_km

In [ ]:
#df.to_excel("GJ12B.xlsx")

## Dataset 2 

In [52]:
#Read the dataset
df2 = pd.read_excel("NL01AA.xltm")

In [53]:
#Exploring dataset contents
df2.head()

,LatLong,Date
0,"13.024994,80.028231",2021-10-04 00:03:00
1,"13.024994,80.028231",2021-10-04 00:08:00
2,"13.024994,80.028231",2021-10-04 00:18:00
3,"13.024994,80.028231",2021-10-04 00:23:00
4,"13.024994,80.028231",2021-10-04 00:28:00


In [54]:
#splitting lat/long col
df2[['lat', 'long']] = df2['LatLong'].str.split(',', 1, expand=True)

In [55]:
# Setting dealer's location 
df2['dealer_lat2'] = 13.070983
df2['dealer_long2'] = 77.452922

In [56]:
# Sample output
df2

,LatLong,Date,lat,long,dealer_lat2,dealer_long2
0,"13.024994,80.028231",2021-10-04 00:03:00,13.024994,80.028231,13.070983,77.452922
1,"13.024994,80.028231",2021-10-04 00:08:00,13.024994,80.028231,13.070983,77.452922
2,"13.024994,80.028231",2021-10-04 00:18:00,13.024994,80.028231,13.070983,77.452922
3,"13.024994,80.028231",2021-10-04 00:23:00,13.024994,80.028231,13.070983,77.452922
4,"13.024994,80.028231",2021-10-04 00:28:00,13.024994,80.028231,13.070983,77.452922
...,...,...,...,...,...,...
524,"13.081451,77.450764",2021-10-05 23:38:00,13.081451,77.450764,13.070983,77.452922
525,"13.081451,77.450764",2021-10-05 23:43:00,13.081451,77.450764,13.070983,77.452922
526,"13.081451,77.450764",2021-10-05 23:48:00,13.081451,77.450764,13.070983,77.452922
527,"13.081451,77.450764",2021-10-05 23:53:00,13.081451,77.450764,13.070983,77.452922


In [59]:
# Checking data types -> lat / long has to be a floating point number
df2.dtypes

LatLong                 object
Date            datetime64[ns]
lat                     object
long                    object
dealer_lat2            float64
dealer_long2           float64
dtype: object

In [60]:
# converting lat/long to floating type

In [61]:
df2["lat"]= df["lat"].astype(float)
df2["long"]= df["long"].astype(float)

In [62]:
# Checking type conversion
df2.dtypes

LatLong                 object
Date            datetime64[ns]
lat                    float64
long                   float64
dealer_lat2            float64
dealer_long2           float64
dtype: object

In [63]:
dealer_lat2 = 13.070983
dealer_lon2 = 77.452922
distances_km = []
for row in df2.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat2, dealer_lon2, row.lat, row.long)
   )

In [64]:
df2['Distance'] = distances_km

In [65]:
df2

,LatLong,Date,lat,long,dealer_lat2,dealer_long2,Distance
0,"13.024994,80.028231",2021-10-04 00:03:00,13.003621,79.990784,13.070983,77.452922,275.02
1,"13.024994,80.028231",2021-10-04 00:08:00,13.003621,79.990784,13.070983,77.452922,275.02
2,"13.024994,80.028231",2021-10-04 00:18:00,13.003621,79.990784,13.070983,77.452922,275.02
3,"13.024994,80.028231",2021-10-04 00:23:00,13.003621,79.990784,13.070983,77.452922,275.02
4,"13.024994,80.028231",2021-10-04 00:28:00,13.003621,79.990784,13.070983,77.452922,275.02
...,...,...,...,...,...,...,...
524,"13.081451,77.450764",2021-10-05 23:38:00,13.357197,77.121357,13.070983,77.452922,47.97
525,"13.081451,77.450764",2021-10-05 23:43:00,13.381352,77.097018,13.070983,77.452922,51.72
526,"13.081451,77.450764",2021-10-05 23:48:00,13.414832,77.081715,13.070983,77.452922,55.46
527,"13.081451,77.450764",2021-10-05 23:53:00,13.442227,77.058349,13.070983,77.452922,59.40


In [66]:
#df2.to_excel("NL01AA.xlsx")

## Dataset 3

In [67]:
df3 = pd.read_excel("NL01AB.xltm")

In [68]:
df3

,Latitude,Longitude,Packet Time
0,13.029557,80.032257,02-Sep-2021 01:05
1,13.029557,80.032257,02-Sep-2021 01:10
2,13.027725,80.029144,02-Sep-2021 01:15
3,13.013268,80.001274,02-Sep-2021 01:20
4,13.002186,80.000305,02-Sep-2021 01:25
...,...,...,...
452,15.841319,74.541664,03-Sep-2021 15:46
453,15.841319,74.541664,03-Sep-2021 15:51
454,15.841319,74.541664,03-Sep-2021 15:56
455,15.841319,74.541664,03-Sep-2021 15:59


In [69]:
df3['Dealer_lat3'] = 15.818516
df3['Dealer_long3'] = 74.494901

In [70]:
df3 = df3[0:455]

In [71]:
df3

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3
0,13.029557,80.032257,02-Sep-2021 01:05,15.818516,74.494901
1,13.029557,80.032257,02-Sep-2021 01:10,15.818516,74.494901
2,13.027725,80.029144,02-Sep-2021 01:15,15.818516,74.494901
3,13.013268,80.001274,02-Sep-2021 01:20,15.818516,74.494901
4,13.002186,80.000305,02-Sep-2021 01:25,15.818516,74.494901
...,...,...,...,...,...
450,15.841319,74.541664,03-Sep-2021 15:36,15.818516,74.494901
451,15.841319,74.541664,03-Sep-2021 15:41,15.818516,74.494901
452,15.841319,74.541664,03-Sep-2021 15:46,15.818516,74.494901
453,15.841319,74.541664,03-Sep-2021 15:51,15.818516,74.494901


In [72]:
df3.dtypes

Latitude        float64
Longitude       float64
Packet Time      object
Dealer_lat3     float64
Dealer_long3    float64
dtype: object

In [73]:
dealer_lat3 = 15.818516
dealer_lon3 = 74.494901
distances_km = []
for row in df3.itertuples(index=False):
   distances_km.append(
       haversine_distance(dealer_lat3, dealer_lon3, row.Latitude, row.Longitude)
   )

In [74]:
df3['Distance'] = distances_km

In [75]:
df3

,Latitude,Longitude,Packet Time,Dealer_lat3,Dealer_long3,Distance
0,13.029557,80.032257,02-Sep-2021 01:05,15.818516,74.494901,672.06
1,13.029557,80.032257,02-Sep-2021 01:10,15.818516,74.494901,672.06
2,13.027725,80.029144,02-Sep-2021 01:15,15.818516,74.494901,671.86
3,13.013268,80.001274,02-Sep-2021 01:20,15.818516,74.494901,669.96
4,13.002186,80.000305,02-Sep-2021 01:25,15.818516,74.494901,670.46
...,...,...,...,...,...,...
450,15.841319,74.541664,03-Sep-2021 15:36,15.818516,74.494901,5.61
451,15.841319,74.541664,03-Sep-2021 15:41,15.818516,74.494901,5.61
452,15.841319,74.541664,03-Sep-2021 15:46,15.818516,74.494901,5.61
453,15.841319,74.541664,03-Sep-2021 15:51,15.818516,74.494901,5.61


In [ ]:
df3.to_excel("NL01AB.xlsx")